#Step 1: Clean and Organize data from `users_consumption_municipio.csv`
 Ideally organize it by tariff type 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
%load_ext rpy2.ipython

**NOTE**
<br>
Remember that the data are in `Official Folder` > `Data` for upload in your environment. Also, all the new datasets we will put in the same folder. 

In [ ]:
###Load the data
%%R
require(tidyverse)
dat<-read_csv("users_consumption_municipio.csv",skip=2,col_types = 
                list(`Cve Edo` = col_double(),
                     `Cve Inegi` = col_number(),
                     `Cve Mun` = col_number(),
                     `Entidad Federativa` = col_character(),
                     Municipio = col_character(),
                     Tarifa = col_character(),
                     `2010` = col_number(),
                     `2011` = col_number(),
                     `2012` = col_number(),
                     `2013` = col_number(),
                     `2014` = col_number(),
                     `2015` = col_number(),
                     `2016` = col_number(),
                     `2017` = col_number()))
summary(dat)

R[write to console]: Loading required package: tidyverse

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: Warning:
R[write to console]:  198092 parsing failures.
  row     col expected actual                              file
13096 Cve Edo a double      A 'users_consumption_municipio.csv'
13097 Cve Edo a double      A 'users_consumption_municipio.csv'
13098 Cve Edo a double      A 'users_consumption_municipio.csv'
13099 Cve Edo a double      A 'users_consumption_municipio.csv'
13100 Cve Edo a double      A 'users_consumption_municipio.csv'
..... ....... ........ ...... ..

    Cve Edo         Cve Inegi       Cve Mun          Entidad Federativa
 Min.   :1.00     Min.   : 1.00   Length:224283      Length:224283     
 1st Qu.:5.00     1st Qu.:14.00   Class :character   Class :character  
 Median :7.00     Median :20.00   Mode  :character   Mode  :character  
 Mean   :6.49     Mean   :19.25                                        
 3rd Qu.:8.00     3rd Qu.:24.00                                        
 Max.   :9.00     Max.   :99.00                                        
 NA's   :198093   NA's   :3                                            
  Municipio            Tarifa               2010           
 Length:224283      Length:224283      Min.   :-9.374e+07  
 Class :character   Class :character   1st Qu.: 0.000e+00  
 Mode  :character   Mode  :character   Median : 0.000e+00  
                                       Mean   : 4.970e+06  
                                       3rd Qu.: 0.000e+00  
                                       Max.   : 1.866e+11  
    

Here, we found that the original table (`users_consumption_municipio.csv`) had two components: The first component is the number of users and the second one is the consumption. So, we will to split the information in order to get a dataset for users and another for consumption. 

In [ ]:
%%R
# split the dataframe
usuarios <- slice(dat,1:112140)
consumo <- slice(dat, 112144:224283)

## Get dataset for users 

In [ ]:
#Assure that there is just 'real States in the dataframe'
%%R
usuarios<-filter(usuarios,`Cve Inegi`<33)

In [ ]:
#Filter for household "Tarifas"
%%R
Tarifas=c("01","1A","1B","1C","1D","1E","1F","DAC")
# Clean NA's and keep just the household "Tarifas"
usuarios<-usuarios %>% 
  filter(!is.na(Tarifa)) %>% 
  filter(Tarifa %in% Tarifas)

In [ ]:
%%R
# Group year variables in the same
usuarios<-usuarios%>% 
  gather(`2010`,`2011`,`2012`,`2013`,
         `2014`,`2015`,`2016`,`2017`,key="year",
         value="users")

In [ ]:
%%R
# Clean for negative values of users and a category of "Total Estatal"
# That is in the "Municipio variable"

usuarios<-usuarios %>% 
  # filter(users>0) %>% 
  filter(Municipio!="Total Estatal")
head(usuarios)

# A tibble: 6 x 8
  `Cve Edo` `Cve Inegi` `Cve Mun` `Entidad Federa… Municipio Tarifa year   users
      <dbl>       <dbl> <chr>     <chr>            <chr>     <chr>  <chr>  <dbl>
1         1           1 001       Aguascalientes   Aguascal… 01     2010  231727
2         1           1 001       Aguascalientes   Aguascal… 1A     2010       0
3         1           1 001       Aguascalientes   Aguascal… 1B     2010       0
4         1           1 001       Aguascalientes   Aguascal… 1C     2010       0
5         1           1 001       Aguascalientes   Aguascal… 1D     2010       0
6         1           1 001       Aguascalientes   Aguascal… 1E     2010       0


In [ ]:
%%R
# Calculate the number of users for each year
n_users<-usuarios %>% 
  group_by(`Cve Inegi`,`Cve Mun`,`Entidad Federativa`,`Municipio`,`Tarifa`,`year`) %>% 
  summarise(users)
head(n_users)

`summarise()` has grouped output by 'Cve Inegi', 'Cve Mun', 'Entidad Federativa', 'Municipio', 'Tarifa'. You can override using the `.groups` argument.
# A tibble: 6 x 7
# Groups:   Cve Inegi, Cve Mun, Entidad Federativa, Municipio, Tarifa [1]
  `Cve Inegi` `Cve Mun` `Entidad Federativa` Municipio      Tarifa year   users
        <dbl> <chr>     <chr>                <chr>          <chr>  <chr>  <dbl>
1           1 001       Aguascalientes       Aguascalientes 01     2010  231727
2           1 001       Aguascalientes       Aguascalientes 01     2011  236475
3           1 001       Aguascalientes       Aguascalientes 01     2012  244474
4           1 001       Aguascalientes       Aguascalientes 01     2013  251805
5           1 001       Aguascalientes       Aguascalientes 01     2014  258658
6           1 001       Aguascalientes       Aguascalientes 01     2015  266481


In [ ]:
%%R
# Calculate the average number of users for the entire period (2010-2017)
avg_users<-n_users %>% 
  group_by(`Cve Inegi`,`Cve Mun`,`Entidad Federativa`,`Municipio`,`Tarifa`) %>%
  summarise(mean_users=mean(users))

head(avg_users)

`summarise()` has grouped output by 'Cve Inegi', 'Cve Mun', 'Entidad Federativa', 'Municipio'. You can override using the `.groups` argument.
# A tibble: 6 x 6
# Groups:   Cve Inegi, Cve Mun, Entidad Federativa, Municipio [1]
  `Cve Inegi` `Cve Mun` `Entidad Federativa` Municipio      Tarifa mean_users
        <dbl> <chr>     <chr>                <chr>          <chr>       <dbl>
1           1 001       Aguascalientes       Aguascalientes 01      255593.  
2           1 001       Aguascalientes       Aguascalientes 1A           1   
3           1 001       Aguascalientes       Aguascalientes 1B           8.12
4           1 001       Aguascalientes       Aguascalientes 1C           0   
5           1 001       Aguascalientes       Aguascalientes 1D           0   
6           1 001       Aguascalientes       Aguascalientes 1E           0   


In [ ]:
# Save the outputs in .csv format
%%R
write_csv(n_users,"n_users.csv")
write_csv(avg_users,"avg_users.csv")

## Get dataset for consumption

In [ ]:
%%R
consumo

# A tibble: 112,140 x 14
   `Cve Edo` `Cve Inegi` `Cve Mun` `Entidad Federati… Municipio   Tarifa  `2010`
       <dbl>       <dbl> <chr>     <chr>              <chr>       <chr>    <dbl>
 1         1           1 001       Aguascalientes     Aguascalie… 01      2.81e8
 2         1           1 001       Aguascalientes     Aguascalie… 1A      0     
 3         1           1 001       Aguascalientes     Aguascalie… 1B      0     
 4         1           1 001       Aguascalientes     Aguascalie… 1C      0     
 5         1           1 001       Aguascalientes     Aguascalie… 1D      0     
 6         1           1 001       Aguascalientes     Aguascalie… 1E      0     
 7         1           1 001       Aguascalientes     Aguascalie… 1F      0     
 8         1           1 001       Aguascalientes     Aguascalie… DAC     2.13e7
 9         1           1 001       Aguascalientes     Aguascalie… 02      9.90e7
10         1           1 001       Aguascalientes     Aguascalie… 03      8.34e5
# …

In [ ]:
#Assure that there is just 'real States in the dataframe'
%%R
consumo <- filter(consumo,`Cve Inegi`< 33)

In [ ]:
%%R
# Clean NA's and keep just the household "Tarifas"
consumo <- consumo %>% 
  filter(!is.na(Tarifa)) %>% 
  filter(Tarifa %in% Tarifas)
head(consumo)

# A tibble: 6 x 14
  `Cve Edo` `Cve Inegi` `Cve Mun` `Entidad Federati… Municipio    Tarifa  `2010`
      <dbl>       <dbl> <chr>     <chr>              <chr>        <chr>    <dbl>
1         1           1 001       Aguascalientes     Aguascalien… 01      2.81e8
2         1           1 001       Aguascalientes     Aguascalien… 1A      0     
3         1           1 001       Aguascalientes     Aguascalien… 1B      0     
4         1           1 001       Aguascalientes     Aguascalien… 1C      0     
5         1           1 001       Aguascalientes     Aguascalien… 1D      0     
6         1           1 001       Aguascalientes     Aguascalien… 1E      0     
# … with 7 more variables: 2011 <dbl>, 2012 <dbl>, 2013 <dbl>, 2014 <dbl>,
#   2015 <dbl>, 2016 <dbl>, 2017 <dbl>


In [ ]:
%%R
# Group year variables in the same
consumo <- consumo %>% 
  gather(`2010`,`2011`,`2012`,`2013`,
         `2014`,`2015`,`2016`,`2017`,key="year",
         value="consumption")

In [ ]:
%%R
# Clean for negative values of users and a category of "Total Estatal"
# That is in the "Municipio variable"

consumo <- consumo %>% 
  # filter(consumption > 0) %>% 
  filter(Municipio!="Total Estatal")

In [ ]:
%%R
# Calculate the number of users for each year
consumption <- consumo %>% 
  group_by(`Cve Inegi`,`Cve Mun`,`Entidad Federativa`,`Municipio`,`Tarifa`,`year`) %>% 
  summarise(consumption)
head(consumption)

`summarise()` has grouped output by 'Cve Inegi', 'Cve Mun', 'Entidad Federativa', 'Municipio', 'Tarifa'. You can override using the `.groups` argument.
# A tibble: 6 x 7
# Groups:   Cve Inegi, Cve Mun, Entidad Federativa, Municipio, Tarifa [1]
  `Cve Inegi` `Cve Mun` `Entidad Federativ… Municipio   Tarifa year  consumption
        <dbl> <chr>     <chr>               <chr>       <chr>  <chr>       <dbl>
1           1 001       Aguascalientes      Aguascalie… 01     2010    281228528
2           1 001       Aguascalientes      Aguascalie… 01     2011    290328904
3           1 001       Aguascalientes      Aguascalie… 01     2012    288575239
4           1 001       Aguascalientes      Aguascalie… 01     2013    293812094
5           1 001       Aguascalientes      Aguascalie… 01     2014    297502527
6           1 001       Aguascalientes      Aguascalie… 01     2015    300805382


In [ ]:
%%R
# Calculate the average number of users for the entire period (2010-2017)
avg_consumption <- consumption %>% 
  group_by(`Cve Inegi`,`Cve Mun`,`Entidad Federativa`,`Municipio`,`Tarifa`) %>%
  summarise(mean_consum =mean(consumption))
head(avg_consumption)

`summarise()` has grouped output by 'Cve Inegi', 'Cve Mun', 'Entidad Federativa', 'Municipio'. You can override using the `.groups` argument.
# A tibble: 6 x 6
# Groups:   Cve Inegi, Cve Mun, Entidad Federativa, Municipio [1]
  `Cve Inegi` `Cve Mun` `Entidad Federativa` Municipio      Tarifa mean_consum
        <dbl> <chr>     <chr>                <chr>          <chr>        <dbl>
1           1 001       Aguascalientes       Aguascalientes 01     296498180. 
2           1 001       Aguascalientes       Aguascalientes 1A           312. 
3           1 001       Aguascalientes       Aguascalientes 1B         54606. 
4           1 001       Aguascalientes       Aguascalientes 1C            77.1
5           1 001       Aguascalientes       Aguascalientes 1D             0  
6           1 001       Aguascalientes       Aguascalientes 1E         64330. 


In [ ]:
#save the outputs in csv.format
%%R
write_csv(consumption,"consumption.csv")
write_csv(avg_consumption,"avg_consumption.csv")

In [ ]:
%%R
# Let's check if users and consumption have the same size 
dim(n_users)

[1] 157312      7


In [ ]:
%%R
dim(consumption)

[1] 157312      7
